
# SLURM: Jobs & Batch Scripts

## What Is A Job?

Fundamentally, a **job**, consists of two things: an allotment of resources & snippets of code to execute using that allotment. 

*This means that when we request that SLURM perform some task (say, executing a snippet of code) we are also requesting a resource allotment which will be reserved for the execution of that task.*

## How Do We Request Resources & Perform Tasks?

There are a couple of ways we can request a resource allotment and perform tasks.  

We might use the command `salloc` to request an allotment (by providing arguments on the command-line) and then interactively run code using the the resources once they become available. Alternatively, if we want for the task to be performed without needing to interact with it, we can use the command `sbatch`.  


Learning the basics of the `sbatch` command is the purpose of this notebook.

### The Command `sbatch`


When provided the path of a shell-script, the `sbatch`
command searches the script for lines beginning with `#SBATCH`.  Any line starting with `#SBATCH` occuring during the preamble (*before any commands*) is treated as meta-data specifying some parameters for a job we're requesting be created.

If these parameters describe a *feasible* job, that job is created and queued until an appropriate resource allotment (matching the provided parameters) can be made.  Once an appropriate allotment becomes available, the provided shell-script is executed using the alloted resources.

---
Any shell-script containing meta-data that `sbatch` can understand *(say, describing a resource request or providing other ambient details)* is called a **batch-script** (or job-script).

---

### **Example 1**

Select the cell below and use `SHIFT`+`ENTER` to create the batch-script `example-one.sh`,

In [ ]:
%%file example-one.sh
#!/bin/bash
### Job Parameters:  
#SBATCH --job-name "My Letter"     # job's name
#SBATCH --output   "letter.%j.log" # output log (%j gets replaced with job's id)
#SBATCH --comment  "A humble job." # comment about job

### Script To Execute:
amount=$((30 + RANDOM % 60)) # number between 30 and 90
memory=$((SLURM_MEM_PER_CPU * SLURM_CPUS_PER_TASK * 1024 * 1024))
draft=(" Dear ${USER},\n\n"
       "  I hope this letter finds you well; it is a glorious $(date +%A)\n"
       "in $(date +%B) and I find myself executing on "$(hostname -s)" with\n"
       "access to ${memory} bytes of memory and $(nproc) cpu(s). Knowing that\n"
       "these resources are shared, I have chosen to surrender ${amount}\n"
       "seconds of my alloted time to processes less fortunate than I.\n\n"
       "Cheers,\n" "  Job ${SLURM_JOB_ID}\n")

# send letter & take a nap.
echo -e "${draft[@]}" && sleep ${amount}
    

#### Submiting The Script & Getting Results

To submit the batch-script to SLURM, we use the `sbatch` command. If `sbatch` successfully creates the job associated with the provided script, it will output the job's id.

Select the following cell and use `SHIFT`+`ENTER` to evaluate it and submit the example batch-script we created,

In [ ]:
sbatch example-one.sh 

Once a job has been successfully submitted, it will appear in the SLURM scheduling queue until it completes. To view jobs currently in the queue, use the command `squeue`. 

Select the following cell and use `SHIFT`+`ENTER` to evaluate it and view the queue (*see if you can locate your job using the JOBID column*)

In [ ]:
squeue

As fun as submitting a job and viewing the queue might be, what happenes once the job executes is fundamentally more important. *Complete the task described in the following cell to view the output of your batch-script,*

In [ ]:
# TASK: determine the name of the file containing the output 
#    generated by the batch-script, put the filename
#    in the quotes below, then execute this cell using SHIFT+ENTER.
cat ""

### **Example 2**

Use `SHIFT`+`ENTER` to create the file `example-two.sh` containing the code from the next cell,

In [ ]:
%%file "example-two.sh"
#!/bin/bash
### Job Parameters:
#SBATCH --job-name  "Arguments" # display name
#SBATCH --output    "out.log"   # where to log terminal output 
#SBATCH --error     "err.log"   #  .. and error messages
#SBATCH --open-mode truncate    # always overwrite log files 

# Resources required
#SBATCH --ntasks 1          # number of tasks we'll perform
#SBATCH --cpus-per-task 1   # num. cpus each task will require
#SBATCH --mem-per-cpu 128   # memory required per cpu (in megabytes)

### Script To Execute:
for item in "$@"; do # loop provided arguments.
    # print the length of the argument (as a string)
    echo -n "The argument '${item}' is ${#item} characters long. "
    # and then do something random...
    amount=$((RANDOM % 6))   # choose `amount` to sleep
    sleep 0.${amount}        # sleep for `amount/10` seconds
    echo "Slept for 0.${amount} seconds."
done

#### Submiting The Script & Getting Results

The awesome thing about batch-scripts is they are also shell-scripts. This means we can execute them without needing to go through SLURM, and that the normal testing & development workflow is available.

Use `SHIFT`+`ENTER` to evaluate the following cell and verify the batch-script works properly,

In [ ]:
bash example-two.sh "first" second{0..2} 

**Optional:** In the cell with batch-script's code, locate the line containing **--cpus-per-task** and change the number of requested CPUs to 2. Save these changes to `example-two.sh` by evaluating that cell, then verify the script still works appropriately.

When you submit a batch-script to SLURM, it's possible to provide a sequence of arguments which will be passed to the script when it executes. 

Use `SHIFT`+`ENTER` to evaluate the following cell and submit the batch-script along with arguments to be used when it executes.

In [ ]:
sbatch example-two.sh "lion" "tigers" argument{1..500}  "bear" 

To verify the  submitted job is running, we'll use `squeue` again.  However, this time, instead of seeing all jobs in the queue, we'll restrict to only jobs associated with our user. To do this we need to use the `--user` argument for `squeue`.

In [ ]:
# excute this cell using SHIFT+ENTER
squeue --user $USER

*How did we know about the `--user` argument?*

We used `man`. The system command `man` is an important resource when using UNIX/Linux based systems.  The command provides detailed documentation (or *manuals*) for various topics, among those topics you'll find "*usage and runtime behavior*" information for a most commands available at a terminal.

In the case of `squeue`, we can learn a lot about what sort of arguments it accepts by skimming the documentation presented by running `man squeue`. For example, a passage detailing how to use the argument `--user` is contained within.

Use `SHIFT`+`ENTER` to evaluate the following cell and display the manual for the `squeue` command. Then, either by, going to the `Edit` menu and selecting `Find`, or using scroll-bar to the right, locate the documentation for the **--users** option,

In [ ]:
man squeue

To finish this example, determine the name of the file containing the output from the batch-script and use the `cat` to display it's contents.

Use `SHIFT`+`ENTER` to evaluate the cell once you have inserted the appropriate command,

### **Example 3**

Use `SHIFT`+`ENTER` to create the file `example-three.sh` containing the code from the next cell,

In [ ]:
%%file "example-three.sh"
#!/bin/bash
### Job Parameters:
#SBATCH --job-name "Recursive"     # job name
#SBATCH --output   "recursive.log" # place to log output
#SBATCH --open-mode append         # always append to logs
#SBATCH --begin     now+10         # specify when the job should start
                                   # .. (waits 10 seconds after submission)
### Script To Execute:
# print job detials
echo "Running job $SLURM_JOB_ID"
echo "Batch script file: $0"

# submit new job using *this* script (recursion)
sbatch "$0"

#### Submiting The Script & Getting Results

We'll submit the job to SLURM as normal, however this time we will request that it be considered for execution immediately. 

In [ ]:
# execute this cell using SHIFT+ENTER
sbatch --begin now example-three.sh

As it turns out, any piece of information which can be provided by an `#SBATCH` line in a batch-script, can be directly passed to `sbatch` as a command-line argument (and vice versa). 

As an example, let's submit a job using the same script, but have the initial job request 1G of memory per cpu, and not start running until after tomorrow at mid-night.

In [ ]:
# execute this cell using SHIFT+ENTER
sbatch --mem-per-cpu 1024 --begin tomorrow example-three.sh 

Let's check the status of the jobs we've submitted. This time, we'll only request information about jobs with the name "Recursive" associated with our user.

In [ ]:
# execute this cell using SHIFT+ENTER
squeue --user $USER --name "Recursive" 

Since these jobs are recursive, they will keep resubmitting themselves *forever*. 

As a result, it's important for us to stop (or *cancel*) any instances which appear in the queue.  To accomplish this, we'll use the command `scancel`. 

This command can handle some of the same arguments as `squeue`. In particular, to cancel all jobs with the name "Recursive" associated with our user, we just run,

In [ ]:
# execute this cell using SHIFT+ENTER
scancel --user $USER --name "Recursive"

Finally, if you want historical information about the jobs you've submitted, you use the `sacct` command. Moreover, it has argments which are similar to those of `squeue`. 

For example, to see a list of jobs (past and present) which have the name "Recursive" and are attached to your user, we can use the following,

In [ ]:
# execute this cell using SHIFT+ENTER
sacct --user $USER --name "Recursive"